In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import cv2
import sys
sys.path.append('../Extract_Features')
# from ipynb.fs.full.<visual_2.ipynb> import *
import import_ipynb
# import_ipynb.sys.path.append('../Extract_Features/visual_2.ipynb')

In [2]:
# imports
import cv2
import os
import numpy as np
import math
from dataclasses import dataclass,fields
from utils import *
from typing import List, Tuple

In [3]:
# TODO: convert_video_to_frames
"""
params : video name 
return : create output directory include the frames of this video 
"""
def convert_video_to_frames(video_name):
    video_path = "../Dataset/"+video_name
    output_dir = "../Dataset/frames"+video_name
    # may be changed
    sampling_rate = 60
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for frame_idx in range(0, total_frames, sampling_rate):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            output_path = os.path.join(output_dir, "frame_{:06d}.jpg".format(frame_idx))
            cv2.imwrite(output_path, frame)
    cap.release()
    return 

In [4]:
# TODO: create classes 
"""
desc: Difference between pixel hue,saturation,luma,edges values of adjacent frames.
"""
from typing import Optional
@dataclass
class delta_feature:
    # by default set all weights to 1
    delta_hue: float = 1.0
    delta_sat: float = 1.0
    delta_lum: float = 1.0
    # TODO: have bigger values that other features,detection threshold may need to be adjusted
    delta_edges: float = 1.0  

"""
desc: features calculated for each frame
"""
@dataclass
class frame_data:
    hue: np.ndarray
    sat: np.ndarray
    lum: np.ndarray
    edges: Optional[np.ndarray]
    hsv_hist: Optional[np.ndarray]
    # TODO: add optical flow

# create defualt weights 
default_delta_feature_weights = delta_feature()
@dataclass
class shot_detector:
    threshold: float = 27.0
    threshold_hist: float = 0.6
    min_scene_len: int = 15
    weights: 'delta_feature' = default_delta_feature_weights
    kernel_size: Optional[int] = None
    last_frame: Optional[frame_data] = None


In [5]:
"""Detect edges  in the frame 
    params:
        lum:  the luma channel of a frame.
        kernel: kernel size 
    return:
        2D 8-bit image where 255--> edge , 0--> other 
"""
import math
def detect_edges(lum: np.ndarray,kernel:int =None) -> np.ndarray:
    if kernel == None:
        # calculate kernel size  depend on the video reselution
        kernel_size = 4 + round(math.sqrt(lum.shape[1]*lum.shape[0]) / 192)
        if kernel_size % 2 == 0:
            kernel_size += 1
        kernel = np.ones((kernel_size, kernel_size), np.uint8) 
    # Estimate levels for thresholding.
    sigma: float = 1.0 / 3.0
    median = np.median(lum)
    low = int(max(0, (1.0 - sigma) * median))
    high = int(min(255, (1.0 + sigma) * median))
    # Calculate edges using Canny algorithm, and reduce noise by dilating the edges.
    # TODO : Implement  Canny 
    edges = cv2.Canny(lum, low, high)
    return cv2.dilate(edges,kernel)

In [6]:
# # TODO: get_HSV_feature_frame
"""
params : frame
return : feature matrix with 3 histograms for H S V
"""
def get_HSV_feature_frame(frame):
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hist_frame = cv2.calcHist(frame, [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist_frame, hist_frame, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    return hist_frame

In [7]:
def get_calcOpticalFlow_feature_frame(frame_pre:np.ndarray, frame:np.ndarray):
    # Compute optical flow and histogram of magnitudes
    rgb1 = cv2.cvtColor(frame_pre, cv2.COLOR_HSV2RGB)
    gray1 = cv2.cvtColor(rgb1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    mag_hist, bins = np.histogram(mag.ravel(), 16, [0, 256])
    mag_hist_norm = mag_hist.astype("float") / mag_hist.sum()
    return mag_hist_norm 

In [8]:
"""
calculate fram score , to compare with the threshold and know if it is shot boundry or not 
params
 frame_num: index of the frame 
 frame_img: frame itself
return 
 frame_score: score for the frame to determine it's boundry or not 
 hist_def:    corelation between the hist of the surrent and prev frame 
"""
def calculate_frame_score(video_param: shot_detector, frame_img: np.ndarray) -> float:
    
    # convert image into HSV colorspace
    hue, sat, lum = cv2.split(cv2.cvtColor(frame_img, cv2.COLOR_BGR2HSV))

    # calculate edges 
    edges = detect_edges(lum) 
    
    # calculate histogram for H S V values 
    hist_hsv = get_HSV_feature_frame(frame_img)


    if video_param.last_frame is None:
        video_param.last_frame = frame_data(hue, sat, lum, edges,hist_hsv)
        return 0.0,1.0
    

    # # calculate histogram for calcOpticalFlow
    # hist_optical = get_calcOpticalFlow_feature_frame([video_param.last_frame.hue, video_param.last_frame.sat, video_param.last_frame.lum], frame_img)
    
    #TODO: calculate HOG
    
    # get score for adjecent frames 
    score_components = delta_feature(
        delta_hue=mean_pixel_distance(hue, video_param.last_frame.hue),
        delta_sat=mean_pixel_distance(sat, video_param.last_frame.sat),
        delta_lum=mean_pixel_distance(lum, video_param.last_frame.lum),
        delta_edges=(0.0 if edges is None else mean_pixel_distance(
            edges, video_param.last_frame.edges)),
    )

    hist_def = hist_compare(hist_hsv,video_param.last_frame.hsv_hist)

    frame_score: float = (
        sum(
            getattr(score_components, field.name) * getattr(video_param.weights, field.name)
            for field in fields(delta_feature)
        )
        / sum(abs(getattr(video_param.weights, field.name)) for field in fields(delta_feature)))
    
    # Store all data required to calculate the next frame's score.
    video_param.last_frame = frame_data(hue, sat, lum, edges,hist_hsv)
    return frame_score,hist_def

In [9]:
"""
return: List of frames where scene cuts have been detected
"""
def process_frame(
    video_param: shot_detector, frames_since_last_cut: List[frame_data],
    frames_count_since_last_cut: int, frame_img: np.ndarray, op) -> Tuple[Optional[float], frame_data]:
    """Returns the shot representation/embeddings (by performing `op`) for a span of frames
    if `frame_img` is a boundary, None otherwise.
    Also returns the `frame_data` for `frame_img` to be used by the caller.
    """
    frame_score ,hist_def= calculate_frame_score(video_param, frame_img)
    # consider any frame over the threshold a new scene, but only if
    # the minimum scene length has been reached (otherwise it is ignored).
    # NOTE: `frames_count_since_last_cut` is the actual number of frames since the last cut, i.e. not sampled every x frames.

    if frames_count_since_last_cut < video_param.min_scene_len:
        return (None, video_param.last_frame,hist_def)
    ##################################################################################################
    # try use the histograme of the boundary not the avg of the frames 
    shot_score = op(frames_since_last_cut) if frame_score >= video_param.threshold else None
    return (shot_score, video_param.last_frame,hist_def)

In [10]:
def avg_frames_features(frames: List[frame_data]) -> frame_data:
    """A reduction operation to perform (averaging) on a list of frame data.
    The reduced data would then be used as a representation of the whole shot.
    """
    assert frames, "Operation can not be performed on 0 frames"
    avg_frame_data = frame_data(0, 0, 0, None)
    count = len(frames)
    attrs = [f.name for f in fields(frames[0])]
    for attr_name in attrs:
        attr_value = sum(getattr(frame, attr_name) for frame in frames) / count
        setattr(avg_frame_data, attr_name, attr_value)
    return avg_frame_data

In [11]:
def get_framesboundary_data():
    video_path = "../Dataset/tears_of_steel_1080p.mov"
    output_dir = "../Dataset/frames"
    #TODO: sampling_rate should be changed 
    sampling_rate = 60
    frame_idx_list = []
    boundary_frames: List[frame_data] = []
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    cap = cv2.VideoCapture(video_path)
    video_try = shot_detector()
    # assert video_try.min_scene_len > sampling_rate, "The sampling rate is must be strictly less than the minimum scene length"
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_data_since_last_cut = []
    
    for frame_idx in range(0, total_frames, sampling_rate):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            output_path = os.path.join(output_dir, "frame_{:06d}.jpg".format(frame_idx))
            cv2.imwrite(output_path, frame)
    
        shot_score, data,hist_def = process_frame(
            video_try, frame_data_since_last_cut,
            len(frame_data_since_last_cut) * sampling_rate,
            frame, avg_frames_features)
        # if shot_score is not None:
        #     print(frame_idx)
        #     # Reset the data accumulator.
        #     frame_data_since_last_cut = []
        # frame_data_since_last_cut.append(data)
        if hist_def < video_try.threshold_hist:
            print(frame_idx)
            boundary_frames.append((data,frame_idx))
    cap.release()
    return boundary_frames

In [12]:
# TODO: get_distance_matrix 
"""
params : 
 feature_vector: array of feature vectors for each shot 
return : 2D distance matrix (Matrix 2D with size N*N ,N is the numer of shots,cell (i,j) have the feature distance between i,j shots)
"""
features_matrix =  np.array([[2, 1, 0, 4, 1], [0, 3, 1, 1, 10],[1, 1, 0, 1, 0]])
def get_distance_matrix(features_matrix):
    D_Matrix = sparse.csr_matrix(features_matrix)
    D_Matrix = cosine_similarity(D_Matrix)
    D_Matrix = 1-D_Matrix
    # D_Matrix.tolist()
    return D_Matrix

#TODO: try eculedian  distance 

In [13]:
"""
params : 
boundary_frames : frames classified to be boundaries thsi is list of tuples (data, frame index)
return : 2D distance matrix (Matrix 2D with size N*N ,N is the numer of shots,cell (i,j) have the feature distance between i,j shots)
"""
def get_distance_matrix(boundary_frames):
    num_shots = len(boundary_frames)
    dist_matrix = np.zeros((num_shots, num_shots))
    for i in range(num_shots):
        print("i",i)
        for j in range(i, num_shots):
            if i == j:
                dist_matrix[i, j] = 0
            else:
                # compute the distance between the i-th and j-th shots' histograms
                dist: float =cv2.compareHist(boundary_frames[i][0].hsv_hist, boundary_frames[j][0].hsv_hist, cv2.HISTCMP_CORREL)
                print(dist)
                dist_matrix[i, j] = 1.0- dist
                dist_matrix[j, i] = 1.0- dist
    return dist_matrix


In [14]:
# D_Matrix = get_distance_matrix(features_matrix)
# print(D_Matrix)
# print(1-D_Matrix)

In [15]:
"""get the total sum of distance of different segmentation 
params :
 Distance_Matrix: 2D distance matrix 
 N: number of shots 
return : table with all different segments in D_Matrix
"""
def get_internal_sums(D_Matrix,N):
    # D_sum = torch.zeros(N,N, device=self.device)
    D_sum = np.zeros(shape=(N,N))
    # initialize diagonal 
    for shot_index in range(N):
        D_sum[shot_index,shot_index] = D_Matrix[shot_index,shot_index]
    # # initialize second diagonal
    # for shot_index in range(0, N-1):
    #     D_sum[shot_index, shot_index+1] = D_Matrix[shot_index:shot_index+1+1, shot_index:shot_index+1+1].sum()
    #     print(D_sum[shot_index, shot_index+1])
    #     # TODO: recheck this condition 
    #     D_sum[shot_index+1, shot_index] = D_sum[shot_index, shot_index+1]
    #     print(D_sum[shot_index+1, shot_index])
    #TODO: if you change the 1 to 2 you should discomment the above code 
    for scene_size in range(1, N):
        for start_shot in range(0, N - scene_size):
            '''
            D_sum[i,j] =+D_sum[i-1,j]                  --> missing last row
                        +D_sum[i,j-1]                  --> missing last column 
                        -D_sum[i-1,j-1]                --> as we sum it twice before
                        +D_Matrix[i,j] + D_Matrix[j,i] -->missing cells in brevious calculations  
            '''
            D_sum[start_shot, start_shot + scene_size] = D_sum[start_shot, start_shot + scene_size - 1] + D_sum[start_shot - 1, start_shot + scene_size] - D_sum[start_shot - 1, start_shot + scene_size - 1] +  D_Matrix[start_shot, start_shot + scene_size] + D_Matrix[start_shot + scene_size, start_shot] 
            # as the matrix is symetric 
            D_sum[start_shot + scene_size, start_shot] = D_sum[start_shot, start_shot + scene_size]
    return D_sum

In [16]:
from operator import itemgetter
# TODO: hierarchical clustering_h_add
# TODO: NOW I AM WORKING WITHOUT USING INTERMEDIATE SUMS (I SHOULD USE IT LATER FOR EFFECIENCY)
"""params : 
    D_matrix : 2D distance matrix (with size N*N)
    scenes_num: Nuber of scenes needed to be generated 
return : T (T is the optimal shot indexes where thw shots are devided into scenes)
"""
def get_optimal_sequence_add1(D_matrix, scenes_num):
    N = D_matrix.shape[0]
    print(N)
    '''
    define cost matrix to define the optimal objective function value ,the optimal one is then cost_Matrix[1,scenes_num]
    cost_Matrix[n,scenes_num] means the cost of deviding  N-n+1 shots to scenes_num scenes
    '''
    cost_Matrix = np.zeros((N, scenes_num))
    print(cost_Matrix.shape)
    boundry_index_Matrix = np.zeros((N, scenes_num), dtype=int)

    # initialization
    for n in range(0, N):
        cost_Matrix[n, 0] = np.sum(D_matrix[n:, n:])
    
    for n in range(0, N):
        boundry_index_Matrix[n, 0] = N - 1

    # build the rest of the table 
    for k in range(1,scenes_num):
        for n in range(0,N):
            diffrent_scene_costs = []
            # define i the index of the next boundry will be betwen (n,N)
            for i in range(n, N):
                if i < N - 1:
                    C_prev = cost_Matrix[i + 1, k - 1]
                else:
                    C_prev = 0
                scene_cost = np.sum(D_matrix[n:i + 1, n:i + 1])
                scene_cost = scene_cost + C_prev
                diffrent_scene_costs.append((scene_cost,i))
            # diffrent_scene_costs = np.array(diffrent_scene_costs)
            # choose the minimum value that we can put i and achieve the least cost for the scene 
            # cost_Matrix[n, k] = np.min(diffrent_scene_costs)
            min_cost = min(diffrent_scene_costs, key=itemgetter(0))
            cost_Matrix[n, k] = min_cost[0]
            # save the index
            # boundry_index_Matrix[n, k] = np.where(diffrent_scene_costs == cost_Matrix[n, k])[0][0] + n
            boundry_index_Matrix[n, k] = min_cost[0]+n
    
    scene_boundries = np.zeros((scenes_num,), dtype=int)
    scene_boundries_prev = 0
    for i in range(0, scenes_num):
        if i == 0:
            scene_boundries_prev = 0
        else:
            scene_boundries_prev = scene_boundries[i - 1]
        print(scene_boundries_prev)
        print(scenes_num - i - 1)
        scene_boundries[i] = boundry_index_Matrix[scene_boundries_prev, scenes_num - i - 1]
    return scene_boundries

In [17]:
frames_boundary= get_framesboundary_data()

240
300
360
420
480
600
1020
1140
1200
1260
1320
1380
1440
1560
1680
1800
1920
1980
2040
2220
2340
2400
2460
2820
3060
3240
3360
3480
3540
3720
4140
4560
4680
4800
4860
4980
5040
5100
5160
5220
5280
5340
5460
5640
5760
6120
6240
6600
6720
6960
7140
7200
7260
7320
7440
7500
7620
7680
7740
7800
7920
7980
8160
8220
8340
8400
8460
8520
8580
8700
8760
8880
8940
9000
9060
9120
9180
9240
9360
9600
9660
9720
9840
9900
9960
10080
10140
10200
10260
10320
10380
10440
10500
10560
10620
10680
11100
11160
11340
11400
11520
11580
11640
11700
11760
11820
11880
11940
12000
12060
12120
12180
12240
12480
12540
12660
12960
13080
13320
13380
13560
13740
13800
14160
17040
17160
17580


In [82]:
print(len(frames_boundary))

127


In [ ]:
dist_matrix = get_distance_matrix(frames_boundary[:40])

In [84]:
D_sum = get_internal_sums(dist_matrix,40)

In [85]:
def get_optimal_sequence_add2(D_matrix, scenes_num):

        shots_num = D_matrix.shape[0]

        if shots_num < 1 or scenes_num < 1 or shots_num != D_matrix.shape[1]:
            print("Error: Problem with input.")
            return []

        if scenes_num > shots_num:
            print("Warning: More scenes than shots. Returning shot boundaries.")
            return np.arrange(1, shots_num + 1)

        if scenes_num == 1:
            return [shots_num - 1]

        D_sum = get_internal_sums(D_matrix,40)

        C = np.zeros((shots_num, scenes_num))
        I = np.zeros((shots_num, scenes_num))

        # initialization
        for nn in range(0, shots_num):
            C[nn, 0] = D_sum[nn, shots_num - 1]
            I[nn, 0] = shots_num - 1

        # fill the rest
        for kk in range(1, scenes_num):
            for nn in range(0, shots_num - kk):
                # T will hold the vector in which we're searching for a minimum
                T = np.transpose(D_sum[nn, nn:shots_num- kk]) + C[nn + 1:shots_num - kk + 1, kk - 1]
                I[nn, kk] = np.argmin(T)
                C[nn, kk] = T[int(I[nn, kk])]
                I[nn, kk] = I[nn, kk] + nn
        # prepare returned boundaries
        boundary_locations = np.zeros(scenes_num)
        the_prev = -1
        for kk in range(0, scenes_num):
            boundary_locations[kk] = I[int(the_prev + 1), scenes_num - kk - 1]
            the_prev = boundary_locations[kk]
        if the_prev != shots_num - 1:
            print("Warning: Encountered an unknown problem.")
        return boundary_locations

In [ ]:
g = get_optimal_sequence_add1(dist_matrix, 3)

In [10]:
# TODO: hierarchical clustering_h_nrm
"""
params : 2D distance matrix (with size N*N)
return : T (T is the optimal shot indexes where thw shots are devided into scenes)
"""

'\nparams : 2D distance matrix (with size N*N)\nreturn : T (T is the optimal shot indexes where thw shots are devided into scenes)\n'

In [11]:
# TODO: get estimated number of scenes

In [87]:
boundary_locations = get_optimal_sequence_add2(dist_matrix,4)

In [88]:
print(boundary_locations)

[ 0.  2. 10. 39.]
